In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

def processar_video_roi(video_path, output_path):

    print("Carregando modelo YOLOv8 Medium (mais preciso)...")
    model = YOLO("yolov8m.pt") 

    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    roi_points = np.array([
        [618, 605], 
        [1137, 600], 
        [1417, 1047], 
        [481, 1042]
    ], np.int32)
    roi_points = roi_points.reshape((-1, 1, 2))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'avc1'), fps, (width, height))

    print(f"Processando: {video_path}...")
    print(f"Resolução: {width}x{height}")

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        results = model.predict(frame, conf=0.3, classes=[0], verbose=False)
        
        boxes = results[0].boxes.xyxy.cpu().numpy()
        pessoas_na_roi = 0

        cv2.polylines(frame, [roi_points], isClosed=True, color=(255, 0, 0), thickness=2)

        for box in boxes:
            x1, y1, x2, y2 = box
            cx = int((x1 + x2) / 2)
            cy = int(y2) 

            is_inside = cv2.pointPolygonTest(roi_points, (cx, cy), False)

            if is_inside >= 0:
                pessoas_na_roi += 1
                # Caixa Verde (Dentro)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            else:
                # Caixa Vermelha (Fora)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)

        # Contador visual
        cv2.rectangle(frame, (20, 20), (350, 80), (0, 0, 0), -1)
        cv2.putText(frame, f"Pessoas na Area: {pessoas_na_roi}", (35, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        out.write(frame)
        
        # Mostra uma versão menor na tela para não ocupar tudo se o vídeo for 4K
        frame_small = cv2.resize(frame, (1024, int(1024 * height / width)))
        cv2.imshow("Monitoramento (Preview Reduzido)", frame_small)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("Concluído! Verifique o arquivo ")

if __name__ == "__main__":
    # ATENÇÃO: Mudei a extensão de saída para .avi
    processar_video_roi("runners.mp4", "resultado_roi.mp4")

Carregando modelo YOLOv8 Medium (mais preciso)...
Processando: runners.mp4...
Resolução: 1920x1080
Concluído! Verifique o arquivo 
